## [Introduction to Cython for Solving Differential Equations](http://hplgit.github.io/teamods/cyode/cyode-sphinx/main_cyode.html) 
-  part three

### Solver for systems of ODEs

In [51]:
import time
import timeit

In [60]:
"""Solve system of ODEs."""

def solver(ode_sys, I, t, integration_method):

    N = len(t)-1
    u = np.zeros((N+1, len(I)))
    u[0, :] = I
    dt = t[1] - t[0]

    for n in range(N):
        u[n+1, :] = integration_method(u[n, :], t[n], dt, n, ode_sys)
    return u, t


def RK2(u, t, dt, n, ode_sys):

    K1 = dt * ode_sys(u, t)
    K2 = dt * ode_sys(u + 0.5 * K1, t + 0.5 * dt)
    unew = u + K2
    return unew


def problem1(u, t):
    return -u + 1.0

from numpy import exp

def problem2(u, t):
    return - u + exp(-2.0 * t)

In [61]:
def run(ode_sys, N, nperiods=40):
    I = np.ones(N)
    time_points = np.linspace(0, nperiods * 2 * np.pi, nperiods * 30 + 1)
    u, t = solver(ode_sys, I, time_points, RK2)

In [62]:
%timeit run(problem1, 1000, 1000)
%timeit run(problem2, 100, 500)

418 ms ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
233 ms ± 79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [67]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython
ctypedef np.float_t DT

cdef extern from "math.h":
    double exp(double)

cdef class Problem:
    cpdef rhs(self, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
                     double t):
        return 0

cdef class Problem1(Problem):
    cpdef rhs(self, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
                     double t):
        return -u + 1 

cdef class Problem2(Problem):
    cpdef rhs(self, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
                     double t):
        return - u + exp(-2*t)

cdef class ODEMethod:
    cpdef advance(self, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
                  double t, double dt, int n, Problem ode_sys):
        return 0

cdef class Method_RK2(ODEMethod):
    cpdef advance(self, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
                  double t, double dt, int n, Problem ode_sys):
        cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] K1, K2, unew
        K1 = dt * ode_sys.rhs(u, t)
        K2 = dt * ode_sys.rhs(u + 0.5 * K1, t + 0.5 * dt)
        unew = u + K2
        return unew
    
# Create names compatible with ode1.py
RK2 = Method_RK2()
problem1 = Problem1()
problem2 = Problem2()

@cython.boundscheck(False) # turn off bounds checking for this func.
@cython.wraparound(False)  # Deactivate negative indexing.
def solver(Problem ode_sys, 
           np.ndarray[DT, ndim=1, mode='c'] I, 
           np.ndarray[DT, ndim=1, mode='c'] t, 
           ODEMethod method):
    
    cdef int N = len(t)-1
    #cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u = np.zeros(N+1, dtype=np.float_t)
    #Cython does not like type specification via dtype when the buffer
    #declares the type
    cdef np.ndarray[DT, ndim=2, negative_indices=False, 
                    mode='c'] u = np.zeros((N+1, len(I)))
    u[0, :] = I   
    dt = t[1] - t[0]
             
    cdef int n
    for n in range(N):
        u[n+1, :] = method.advance(u[n, :], t[n], dt, n, ode_sys)
    
    return u, t


In [65]:
%timeit run(problem1, 1000, 1000)
%timeit run(problem2, 100, 500)

552 ms ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
226 ms ± 78.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# simple using functions

In [74]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython
ctypedef np.float_t DT

cdef extern from "math.h":
    double exp(double)

@cython.boundscheck(False) # turn off bounds checking for this func.
@cython.wraparound(False)  # Deactivate negative indexing.    
cpdef solver(ode_sys, np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] I, 
             np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] t, 
             integration_method):

    cdef int N = len(t)-1
    cdef np.ndarray[DT, ndim=2, negative_indices=False, 
                    mode='c'] u = np.zeros((N+1, len(I)))
    u[0, :] = I
    cdef double dt = t[1] - t[0]
    cdef int n
    for n in range(N):
        u[n+1, :] = RK2(u[n, :], t[n], dt, n, ode_sys)
    return u, t


def RK2(np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, 
        double t, double dt, int n, ode_sys):
    
    cdef np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] K1, K2, unew
    K1 = dt * problem1(u, t)
    K2 = dt * problem1(u + 0.5 * K1, t + 0.5 * dt)
    unew = u + K2
    return unew

cdef problem1(np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, double t):
    return -u + 1.0

cdef problem2(np.ndarray[DT, ndim=1, negative_indices=False, mode='c'] u, double t):
    return - u + exp(-2.0 * t)

In [73]:
%timeit run(problem1, 1000, 1000)
# %timeit run(problem2, 100, 500)

424 ms ± 8.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
